In [1]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import random
import cv2
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from imutils import paths
import random # Add this line to import the random module

imagePaths = sorted(list(paths.list_images('/content/drive/MyDrive/rice_leaf_diseases')))

random.seed(42)
random.shuffle(imagePaths)

data = []
labels = []
image_dims = (224, 224, 3)

In [4]:
for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (image_dims[1], image_dims[0]))
    image = img_to_array(image)
    data.append(image)
    l = label = imagePath.split(os.path.sep)[-2].split("_")
    labels.append(l)

data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
print("{} images ({:.2f}MB)".format(len(imagePaths), data.nbytes / (1024 * 1000.0)))

120 images (141.12MB)


In [5]:
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)

# Define the MobileNetV2 model
def MobileNetV2_model(learning_rate, input_shape, class_number):
    baseModel = MobileNetV2(include_top=False, input_tensor=Input(shape=input_shape))
    for layer in baseModel.layers[:-4]:
        layer.trainable = False

    model = Sequential()
    model.add(baseModel)
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(50, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(class_number, activation='softmax'))
    return model

In [6]:
bs = 32
lr = 0.0001
shape = (224, 224, 3)
epochs = 10
class_number = 3

In [7]:
model = MobileNetV2_model(lr, shape, class_number)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

# Split the data
trainX, testX, trainY, testY = train_test_split(data, labels, test_size=0.20)

# Train the model
print("[INFO] training ...")
model.fit(trainX, trainY, batch_size=32, steps_per_epoch=len(trainX) // 32, validation_data=(testX, testY),
          validation_steps=len(testX) // 32, epochs=epochs)

# Evaluate the model
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=32)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testY.argmax(axis=1), predIdxs, target_names=mlb.classes_))

# ** Save the model to HDF5 format for conversion to TensorFlow.js **
model.save('mobilenet_model.h5')
print("[INFO] Model saved as mobilenet_model.h5")


<ipython-input-5-8ffec86a2674>:6: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  baseModel = MobileNetV2(include_top=False, input_tensor=Input(shape=input_shape))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
[INFO] training ...
Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - accuracy: 0.3086 - loss: 4.4278 - val_accuracy: 0.3750 - val_loss: 1.9216
Epoch 2/10


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 279ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.3750 - val_loss: 1.9216
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.6549 - loss: 2.0542 - val_accuracy: 0.8333 - val_loss: 0.3367
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 284ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8333 - val_loss: 0.3367
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.6302 - loss: 1.9766 - val_accuracy: 0.7083 - val_loss: 0.6191
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 280ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7083 - val_loss: 0.6191
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.7930 - loss: 1.1441 - val_accuracy: 0.7917 - val_loss: 0.4385
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 279ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7917 - val_loss: 0.4385
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.7500 - loss: 0.9304 - val_accuracy: 0.7500 -

                       precision    recall  f1-score   support

Bacterial leaf blight       0.89      0.89      0.89         9
           Brown spot       0.78      0.70      0.74        10
            Leaf smut       0.50      0.60      0.55         5

             accuracy                           0.75        24
            macro avg       0.72      0.73      0.72        24
         weighted avg       0.76      0.75      0.75        24

[INFO] Model saved as mobilenet_model.h5


In [8]:
!pip install tensorflowjs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 95.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1


In [9]:
model.save('mobilenet_model.h5')

In [10]:
!tensorflowjs_converter --input_format keras /content/mobilenet_model.h5 /content/tfjs_model

2024-09-27 11:38:26.834543: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-27 11:38:26.857517: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-27 11:38:26.864732: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-27 11:38:28.141987: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
failed to lookup keras version from the file,
    this is likely a weight only file
